In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

In [3]:
# Define paths
train_dir = "/kaggle/input/covid19-image-dataset/Covid19-dataset/train"
test_dir = "/kaggle/input/covid19-image-dataset/Covid19-dataset/test"

In [4]:
# Define classes
classes = ["Covid", "Normal", "Viral Pneumonia"]

In [5]:
# Print number of images in each class
for class_name in classes:
    train_class_dir = os.path.join(train_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)
    print(f"Train {class_name}: {len(os.listdir(train_class_dir))} images")
    print(f"Test {class_name}: {len(os.listdir(test_class_dir))} images")

Train Covid: 111 images
Test Covid: 26 images
Train Normal: 70 images
Test Normal: 20 images
Train Viral Pneumonia: 70 images
Test Viral Pneumonia: 20 images


In [6]:
# Data Preparation
img_size = (2746, 2382)
batch_size = 8

In [7]:
train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 251 images belonging to 3 classes.


In [9]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 66 images belonging to 3 classes.


In [10]:
# Model Architecture
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(2746, 2382, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(3, activation='softmax')(x)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
model = Model(inputs=base_model.input, outputs=output)

In [12]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [13]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Model Training
epochs = 20
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs
)

Epoch 1/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 62s 2s/step - accuracy: 0.8970 - loss: 0.2808
Epoch 2/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 387us/step - accuracy: 1.0000 - loss: 0.1170
Epoch 3/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.9056 - loss: 0.2871
Epoch 4/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 382us/step - accuracy: 0.8750 - loss: 0.1524
Epoch 5/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.9221 - loss: 0.2440
Epoch 6/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 373us/step - accuracy: 0.8750 - loss: 0.4338
Epoch 7/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 59s 1s/step - accuracy: 0.8849 - loss: 0.2409
Epoch 8/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 374us/step - accuracy: 1.0000 - loss: 0.0770
Epoch 9/20
 3/31 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.9097 - loss: 0.2599

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()